# Exercise 5: Using basic data science to understand our graph

## Introduction

We now know how to create graphs and do some basic querying and manipulation on them.  In this exercise we are going to get a bit more complicated.  In particular, we are going to explore some techniques to calculate the following:

- Node similarity
- PageRank
- Community detection

We will be working with the Wikidata knowledge graph to do this.  If you already have a Sandbox instance created with that graph, you can use that.  Otherwise there are cells below to get that built back up on a new instance.

## Graph Data Science Library

Similar to APOC, we will be using a different library to do these calculations: the [Graph Data Science Library](https://dev.neo4j.com/graph_data_science).  There are built-in functions to achieve each of these tasks that make things significantly easier.

So let's begin in the usual way...

In [ ]:
from neo4j import GraphDatabase

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [ ]:
uri = ''
user = 'neo4j'
pwd = ''

conn = Neo4jConnection(uri=uri, user=user, pwd=pwd)

conn.query("MATCH (n) RETURN COUNT(n)")

## Populate the database

This next cell is not necessary if you already have the database of the Barack Obama Wikidata graph going.

In [ ]:
conn.query("CALL apoc.import.json('https://raw.githubusercontent.com/cj2001/nodes2021_kg_workshop/main/json_files/wiki.json')")
conn.query("MATCH (n) RETURN COUNT(n)")

# Delete duplicate nodes

query = """MATCH (n:Node) 
           WITH n.name AS name, COLLECT(n) AS nodes 
           WHERE SIZE(nodes)>1 
           FOREACH (el in nodes | DETACH DELETE el)
"""

conn.query(query)

# Populate node labels

query1 = """MATCH (n:Node) 
            SET n.type_ls = apoc.convert.toStringList(n.type)
"""

query2 = """MATCH (n:Node) 
            CALL apoc.create.addLabels(n, n.type_ls) 
            YIELD node 
            RETURN COUNT(node)
"""

conn.query(query1)
conn.query(query2)

## _EXERCISE:_ Calculate in-degree of the nodes

We could infer the importance of a node based on how many other nodes link to it.  See if you can identify the most important nodes.

_Hint:_ You might want to check out the `size()` function in Cypher

In [ ]:
# your code here

## In-memory graphs

The fundamental building block of the GDS library is the in-memory graph.  These are "subgraphs" created of the entire database on which we will do our calculations.  We will select the specific node and edge types for our calculations using them.

You might ask why we don't just use the whole graph?  There are a few reasons for that:

1. As the graphs get big, this will become prohibitive.
2. Many graph algorithms require undirected and/or monopartite graphs.  Using in-memory graphs allows us to prepare the correct data for the subsequent calculations.

There are two ways to create an in-memory graph with GDS: native projections and Cypher projections.  The former is much faster, the later is a bit more flexible.  You will definitely want to consult the [documentation](https://neo4j.com/docs/graph-data-science/current/management-ops/) on this because the syntax can get involved.

### What in-memory graph should we create?

Well, let's start with a hypothesis.  I am going to hypothesize that the humans in the graph who go to the same university will be similar.  So we are going to create an in-memory graph exploring which nodes of type "private university" or "graduate school" the humans are connected to.  We will use the following query to do this:

```
CALL gds.graph.create('human-edu', ['human', 'private university', 'graduate school'], '*')
```

This is a basic native projection graph.  We are essentially specifying three things:

1. A name of the graph (`human-edu`)
2. The nodes that will be involved (`['human', 'private university', 'graduate school']`)
3. The edges that will be involved (`'*'`)

That is pretty much the formula to creating all of the graphs, but we will make them a little bit more complicated as we go, just for educational purposes. 

So let's get started!

In [ ]:
query = """CALL gds.graph.create('human-edu', ['human', 'private university', 'graduate school'], '*')"""

conn.query(query)

## Node similarity calculation

Now that we have our in-memory graph, we are going to use the [Node Similarity](https://neo4j.com/docs/graph-data-science/current/algorithms/node-similarity/) function to look at which humans are similar.  This function is essentially Jaccard Similarity based on which nodes each have in common.

In [ ]:
query = """CALL gds.nodeSimilarity.stream('human-edu')
           YIELD node1, node2, similarity
           RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity
           ORDER BY similarity DESCENDING, Person1, Person2
"""

conn.query(query)

## _EXERCISE:_ Node similarity based on awards received

Repeat the above process, but look at similarity based on the awards the humans received (i.e. the `award_received` relationship).  You will need to construct a new in-memory graph and then run the node similarity function.

In [ ]:
# your code here

## PageRank

<img src="images/pagerank.png" width="400">

We discussed the PageRank algorithm earlier in this course.  But we will now use our `human-edu` graph to calculate it.  As you will see, based on our knowledge of politicians it is not very surprising.

In [ ]:
query = """CALL gds.pageRank.stream('human-edu')
           YIELD nodeId, score
           RETURN gds.util.asNode(nodeId).name AS name, score
           ORDER BY score DESC, name ASC
           LIMIT 10
"""

conn.query(query)

## Community detection

There are several ways we can do [community detection](https://neo4j.com/docs/graph-data-science/current/algorithms/community/).  We covered some of them earlier in this course.  For this calculation, we will use the popular [Louvain Method](https://en.wikipedia.org/wiki/Louvain_method).  In this approach, [the algorithm](https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/) identifies communities through comparisons of community density.  It is an iterative approach where the algorithm tries various groupings of community until an optimum is reached.

Again, we will start with a hypothesis to create our in-memory graph.  Let's hypothesize that communities from around members with other entities (i.e. the `member_of` relationship).  We might be considering humans, organizations, countries, or any other type of node.  So we are going to this time create an in-memory graph of all nodes that have the `member_of` relationship.  We also will be creating the in-memory graph as an undirected graph.  Many of the GDS algorithms work best on undirected graphs, Louvain being one of them.

In [ ]:
query = """CALL gds.graph.create( 'member-undir', 
                                  '*', 
                                  { MEMBER: 
                                       { type: 'member_of', 
                                         orientation: 'UNDIRECTED' 
                                       } 
                                   } 
                                ) 
           YIELD graphName, nodeCount, relationshipCount
"""

conn.query(query)

In [ ]:
query = """CALL gds.louvain.stream('member-undir') 
           YIELD nodeId, communityId 
           RETURN COUNT(gds.util.asNode(nodeId).name) AS total, communityId 
           ORDER BY total DESC
           LIMIT 10
"""

conn.query(query)

## _EXERCISE:_ Do the larger communities make sense?

Most of our communities will have 1 node in them, which is not particularly informative.  Using the above queries, identify which communities have the most nodes attributed to them and see if the largest one makes sense.

In [ ]:
# your code here